# Tutorial 13: Ingestion Agent Workflows

Learn how to:
- Configure `DataIngestionAgent` to watch a directory
- Trigger ingestion runs and inspect activity
- Verify newly ingested tensors

When the API is offline, this notebook simulates agent behavior.

In [ ]:
from tutorial_utils import ping_server, ensure_dataset, fetch_dataset, pretty_json
from pathlib import Path
import json
import numpy as np
import requests

API = "http://127.0.0.1:7860"
SERVER = ping_server(API)
print(f"\ud83d\udce1 Tensorus server available: {SERVER}")

DATASET = "ingestion_demo"
WATCH_DIR = Path("temp_ingestion_source_api")
WATCH_DIR.mkdir(parents=True, exist_ok=True)

if SERVER:
    print(pretty_json(ensure_dataset(DATASET, API).payload))
else:
    print("Demo mode: dataset + directory assumed ready.")


## Seed new files for ingestion
The agent polls `temp_ingestion_source_api/` and converts files into tensors.

In [ ]:
payload = {"matrix": np.random.rand(4, 4).tolist(), "metadata": {"sensor": "imu"}}
sample_file = WATCH_DIR / "demo_tensor.json"
sample_file.write_text(json.dumps(payload, indent=2))
print(f"Wrote sample file: {sample_file}")


## Start the agent (manual step)
Run the following command in a terminal to launch the ingestion agent.

```bash
python tensorus/ingestion_agent.py --dataset ingested_data_api --source temp_ingestion_source_api
```

The notebook polls for new records; demo mode synthesizes mock responses.

In [ ]:
if SERVER:
    try:
        records = fetch_dataset(DATASET, API)
    except requests.RequestException as exc:
        records = {"error": str(exc)}
else:
    records = {"demo_mode": True, "data": [{"metadata": {"source": "demo_file"}, "record_id": "demo-1"}]}

print(pretty_json(records))


## Cleanup
Remove temporary files created for the demonstration.

In [ ]:
sample_file.unlink(missing_ok=True)
print("Temporary file removed.")
